In [1]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
        
clearall()

In [2]:
############
#### 50 PIPS
###########

In [15]:
############################
######## LIBRAIRIES ########
############################
print('Importing Librairies...')
from sklearn.linear_model import LinearRegression
import colorama as col
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns
import time
import os
import datetime as dt
import numpy as np
import fxcmpy
import socketio
import shelve
import talib
import statistics
import pyttsx3

#from numpy import loadtxt
#from functools import reduce

number = 10000
_period = 'm5'
_period2 = 'H1'
_period3 = 'D1'
_total = 0
_cash_ini = 200000
_cash = _cash_ini
_rate = 1
#_flag = 0

DATE = []
CONTRACT = []
OPEN_POZ = []
CLOSE_POZ = []
RATE_OPEN_POZ = []
RATE_CLOSE_POZ = []
PNL_LAT = []
PNL_REAL = []
TOTAL_OPEN = []
TOTAL_CLOSE = []
PRICE_BUY = []
PRICE_SELL = []
DER_POZ = []
TOTAL_PNL_LAT = []
TOTAL_PNL_REAL = []
_cash = _cash_ini
WINNERS = []
LOOSERS = []
SIGNAL = []
OPEN = []
OPEN_BID = []
OPEN_ASK = []


_atr = 80
_year_bottom = '2018-01-01 00:00:01'
_year_top = '2018-12-31 23:59:59'

_verbose = 1

engine = pyttsx3.init()

print('Librairies imported\n')
print('version fxcmpy :',fxcmpy.__version__)
%matplotlib inline
plt.style.use('seaborn')
engine.say("librairie loaded")
engine.runAndWait()

df_resultats = pd.DataFrame(index=['TimeFrame 1','TimeFrame2','Equity','Nbre Winners','Nbre Loosers','Max lenght of trade','Min lenght of trade',\
    'Average lenght of trade','Cumul pnl'])

x = 'EUR/USD'

print('Prêt')

Importing Librairies...
Librairies imported

version fxcmpy : 1.2.6
Prêt


In [16]:
%%time

engine.say("קדימה")
engine.runAndWait()

position = 0

print(col.Fore.MAGENTA,'Le rate du ticker',x,'est à ',_rate,col.Style.RESET_ALL)

print('Chargement des bases')

data = pd.read_csv('Base_Clean/'+x.replace('/','')+'m5.csv')
data2 = pd.read_csv('Base_Clean/'+x.replace('/','')+'D1.csv')

data.index = pd.to_datetime(data['Date'])
data = data.drop(['Date','Total'],axis=1)
data.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data2['Date'] = data2['Date'].shift(-1)
data2.index = pd.to_datetime(data2['Date'])
data2 = data2.drop(['Date','Total'],axis=1)
data2.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data = data[(data.index >= _year_bottom) & (data.index <= _year_top)]
data2 = data2[(data2.index >= _year_bottom) & (data2.index <= _year_top)]

price = data.copy()
price2 = data2.copy()

print('Bases chargées')
print('Préparation des dataframes')

if '/JPY' in x:
    _pip_factor = 100
else:
    _pip_factor = 10000
price2['Range_Bid'] = (price2.Open_Bid.shift(1) - price2.Close_Bid.shift(1)) * _pip_factor
price2['Range_Ask'] = (price2.Open_Ask.shift(1) - price2.Close_Ask.shift(1)) * _pip_factor
price2['Range'] = (price2.Open.shift(1) - price2.Close.shift(1)) * _pip_factor
price2['OpenBid_Shift'] = price2['Open_Bid'].shift(-1)
price2['OpenAsk_Shift'] = price2['Open_Ask'].shift(-1)
price2['Open_Shift'] = price2['Open'].shift(-1)
price2['Signal'] = np.where((price2.Range_Ask >= _atr) | (price2.Range_Bid >= _atr) | (price2.Range >= _atr),1,\
    np.where((price2.Range_Ask <= -_atr) | (price2.Range_Bid <= -_atr) | (price2.Range <= -_atr),-1,0))
price2['TimeRange']= [price2.index[i].strftime(format='%Y-%m-%d') for i in range(len(price2))]
price['TimeRange']= [price.index[i].strftime(format='%Y-%m-%d') for i in range(len(price))]

print('TETEL process effectué')

print(col.Fore.CYAN,'ENTERING THE BACKTEST',col.Style.RESET_ALL)

for i in range(len(price)):
    try:
        SIGNAL.append(price2[price2.TimeRange == price.TimeRange[i]].Signal.values[0])
        OPEN.append(price2[price2.TimeRange == price.TimeRange[i]].Open_Shift.values[0])
        OPEN_BID.append(price2[price2.TimeRange == price.TimeRange[i]].OpenBid_Shift.values[0])
        OPEN_ASK.append(price2[price2.TimeRange == price.TimeRange[i]].OpenAsk_Shift.values[0])
    except:
        SIGNAL.append(999)
        OPEN.append(999)
        OPEN_BID.append(999)
        OPEN_ASK.append(999)
price['Signal'] = SIGNAL
price['OpenBid_Shift'] = OPEN_BID
price['OpenAsk_Shift'] = OPEN_ASK
price['Open_Shift'] = OPEN

price = price[(price.Signal != 999) & (price.OpenBid_Shift != 999) & (price.OpenAsk_Shift != 999)]
price = price.dropna()

_position = 0
_equity = 0
_nbtransactions = 0
backtest_graph = pd.DataFrame()
EQUITY = [_cash]
CASH = [_cash]
_winner = 0
_looser = 0
_index_entry = 0
TRADE_DURATION = []
_average_duration = 0
_size = 5000

PRICE_BUY = []
PRICE_SELL = []

_total = 0

_target = 0.006
_sl = 0.0015
_spread_filter = 0.00050
_filtre_multipoz = 0.5 # En %
_filtre_multipoz = _filtre_multipoz/100

_open_buy = 0
_open_sell = 0
_minipoz = 0

for i in range(0,len(price)):
    
    if _position == 0 :
        # BUY SIGNAL
        if price.Signal[i] == -1 : # -1 and (price.OpenBid_Shift[i] - price.High_Bid[i]) * _pip_factor < -15 and \
            #(price.OpenBid_Shift[i] - price.High_Bid[i]) * _pip_factor > -35 :
            _pnl = 0
            _open_buy += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _position = 1
            _index_entry = i
            _nbtransactions += 1
            price_buy = price.Close_Ask.iloc[i]
            PRICE_BUY.append(price_buy)
            _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 bought at', price_buy,'(verification liste',PRICE_BUY[-1],')')

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_buy)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_OPEN.append(1) 
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 

        
        
        # SELL SIGNAL
        elif price.Signal[i] == 1 : # 1 and (price.OpenAsk_Shift[i] - price.Low_Ask[i]) * _pip_factor > 15 and\
            #(price.OpenAsk_Shift[i] - price.Low_Ask[i]) * _pip_factor < 35 :
            _pnl = 0
            _open_sell += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _index_entry = i
            _position = -1
            _nbtransactions += 1
            price_sell = price.Close_Bid.iloc[i]
            PRICE_SELL.append(price_sell)
            _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(-1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_sell)
            RATE_CLOSE_POZ.append(0) 
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0)

            TOTAL_OPEN.append(1)

        else :
            _pnl = 0
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
    
    elif _position == 1:

        if price.Signal[i] == -1 : # -1 and (price_buy - price.Close_Bid[i])/price_buy>=_filtre_multipoz:
        
            _pnl = (price.Close_Ask.iloc[i] - _price_buy_mean) * _size * _open_buy
            _open_buy += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _position = 1
            #_index_entry = 0
            _nbtransactions += 1
            price_buy = price.Close_Ask.iloc[i]
            PRICE_BUY.append(price_buy)
            _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position',_open_buy,'bought at ', price_buy,'(verification liste',PRICE_BUY[-1],')')
                print(col.Fore.YELLOW,'Nouvelle moyenne du price_buy',col.Style.RESET_ALL,_price_buy_mean)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_buy)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_OPEN.append(1) 
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 


        elif (float(price.Low_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) >= _target \
            or ((float(price.Low_Bid.iloc[i]) - float(_price_buy_mean) > 0) and (i - _index_entry > 200)):#slowd[i]>80:
    
            _position = 0
            _pnl = (price.Low_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _winner += _open_buy

            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl > 0:
                    print(_open_buy,'positions TP closed at',price.Low_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                else:
                    print(_open_buy,'positions TP closed at',price.Low_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Low_Bid.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_buy)
            PRICE_BUY = [] 
            _open_buy = 0

        elif ((float(price.Low_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) <= -_sl) or \
            ((i - _index_entry) > 50 and (float(price.Low_Bid.iloc[i]) - float(_price_buy_mean)) < 0):
            _position = 0
            _pnl = (price.High_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_buy

            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl < 0:
                    print(_open_buy,'positions SL closed at',price.High_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                else :
                    print(_open_buy,'positions SL closed at',price.High_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.High_Bid.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_buy) 
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            PRICE_BUY = []
            _open_buy = 0
        else:

            _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(0) ############### 
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 


    elif _position == -1:
        
        if price.Signal[i] == 1 : # 1 and (price.Close_Bid[i] - price_sell)/price_sell>=_filtre_multipoz :
            
            _pnl = - (price.Low_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _open_sell += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _position = -1
            #_index_entry = 0
            _nbtransactions += 1
            price_sell = price.Close_Bid.iloc[i]
            PRICE_SELL.append(price_sell)
            _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')
                print(col.Fore.YELLOW,'Nouvelle moyenne du price_sell',col.Style.RESET_ALL,_price_sell_mean)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_sell)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_OPEN.append(1) 
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 



        elif (float(price.High_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) <= -_target \
            or ((float(price.High_Ask.iloc[i]) - float(_price_sell_mean) < 0) and (i - _index_entry > 200)):
            _position = 0
            _pnl = - (price.High_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _winner += _open_sell
            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl > 0 :
                    print(_open_sell,'positions TP closed at',price.High_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                else:
                    print(_open_sell,'positions TP closed at',price.High_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(-1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.High_Ask.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_sell)
            PRICE_SELL = [] 
            _open_sell = 0

        elif ((float(price.Low_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) >= _sl) or \
            ((i - _index_entry) > 50 and (float(price.Low_Ask.iloc[i]) - float(_price_sell_mean)>0)) :
            _position = 0
            _pnl = - (price.Low_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_sell
            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                if _pnl < 0 :    
                    print(_open_sell,'position SL closed at',price.Low_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                else:
                    print(_open_sell,'position SL closed at',price.Low_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(-1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Low_Ask.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_sell)
            PRICE_SELL = []
            _open_sell = 0

        else:

            _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _equity = _cash + _pnl

            EQUITY.append(_equity)
            CASH.append(_cash)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_PNL_LAT.append(_pnl)

            TOTAL_PNL_REAL.append(0)
            
    elif i == (len(price)-1) and _position != 0 :

        if _position == -1:
            _position = 0
            _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_sell
            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_sell,'position closed at',price.Close_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(-1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_sell)
            PRICE_SELL = []
            _open_sell = 0

        if _position == 1:

            _position = 0
            _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_buy

            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_buy,'positions closed at',price.Close_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_buy) 
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            PRICE_BUY = []
            _open_buy = 0

try:
    _average_duration = round(statistics.mean(TRADE_DURATION),2)
    _max_duration = max(TRADE_DURATION)
    _min_duration = min([item for item in TRADE_DURATION if item !=0])

except:
    print("(pas de duration)") 
    _average_duration = 'NA' 
print(col.Fore.BLUE,'Pour le ticker',col.Fore.YELLOW,x,col.Style.RESET_ALL)
if _total > 0:              
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.GREEN,round(_total,2),'!. En ',\
        _nbtransactions,col.Style.RESET_ALL,' transactions.' )
else:
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.RED,round(_total,2),'!. En ',\
        _nbtransactions,col.Style.RESET_ALL,' transactions.' ) 
print(col.Fore.GREEN,"\nNbre de winners :",_winner,col.Style.RESET_ALL)
print(col.Fore.RED,"\nNbre de loosers :",_looser,col.Style.RESET_ALL)

backtest_graph['Equity'] = EQUITY

df_resultats[x] = [_period,_period2,(round(_equity,2)),(_winner),(_looser),(_max_duration),(_min_duration),(_average_duration),(_total)]

DER_POZ.append(_pnl)

engine.say("Finito caucau")
engine.runAndWait()




 Le rate du ticker EUR/USD est à  1 
Chargement des bases
Bases chargées
Préparation des dataframes
TETEL process effectué
 ENTERING THE BACKTEST 
 2018-01-14 19:20:00 
Position 1 bought at 1.22061 (verification liste 1.22061 )
 2018-01-14 19:40:00 
Position 2 bought at  1.22061 (verification liste 1.22061 )
 Nouvelle moyenne du price_buy  1.22061
 2018-01-14 20:05:00 
Position 3 bought at  1.22045 (verification liste 1.22045 )
 Nouvelle moyenne du price_buy  1.22056
 2018-01-14 20:10:00 
Position 4 bought at  1.22045 (verification liste 1.22045 )
 Nouvelle moyenne du price_buy  1.22053
 2018-01-14 20:25:00 
Position 5 bought at  1.22007 (verification liste 1.22007 )
 Nouvelle moyenne du price_buy  1.22044
 2018-01-14 20:30:00 
Position 6 bought at  1.2200799999999998 (verification liste 1.2200799999999998 )
 Nouvelle moyenne du price_buy  1.22038
 2018-01-14 20:35:00 
Position 7 bought at  1.22022 (verification liste 1.22022 )
 Nouvelle moyenne du price_buy  1.22036
 2018-01-14 20:40:

In [17]:
df_historical = pd.DataFrame()
df_historical = pd.DataFrame(index=DATE)
df_historical['Contract'] = CONTRACT
df_historical['Open_Poz'] = OPEN_POZ
df_historical['Close_Pos'] = CLOSE_POZ
df_historical['Rate_Open_Poz'] = RATE_OPEN_POZ 
df_historical['Rate_Close_Poze'] = RATE_CLOSE_POZ
df_historical['Pnl_Lat'] = TOTAL_PNL_LAT
df_historical['Pnl_Real'] = TOTAL_PNL_REAL
df_historical = df_historical.sort_index()
_generated_cash = round(df_historical.Pnl_Real.sum() * _rate,2)
_generated_cash_perc = round((_generated_cash / _cash_ini) * 100,2)
print(col.Fore.YELLOW,x,col.Fore.BLUE,'results',col.Style.RESET_ALL)
print(col.Fore.MAGENTA,'Tested Period',_year_bottom,' à',_year_top,col.Style.RESET_ALL)
print(col.Fore.CYAN,'Total Number of trades',max([sum(TOTAL_OPEN),sum(TOTAL_CLOSE)]),col.Style.RESET_ALL)
if _generated_cash <= 0:
    print('Started Cash :',_size)
    print('P&L in currency:',col.Fore.RED,str(_generated_cash)+'$',col.Style.RESET_ALL)
    print('P&L in %:',col.Fore.RED,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

else:
    print('Started Cash :',_size)
    print('P&L  in currency:',col.Fore.GREEN,str(_generated_cash)+'$',col.Style.RESET_ALL)
    print('P&L in %:',col.Fore.GREEN,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

#print('Pnl latent minimal :',str(round((df_historical.Pnl_Real.min()) * _rate,2))+'$')
#print('Pnl latent maximal :',str(round((df_historical.Pnl_Real.max())* _rate,2))+'$')
#print('Pnl latent moyen :',str(round(df_historical.Pnl_Real.mean()* _rate,2))+'$')
#print('Maximal trade duration',_max_duration)
#print('Minimal trade duration',_min_duration)
print('Average trade duration',_average_duration)
print('# Winners ',df_resultats.T['Nbre Winners'].sum())
print('# Loosers ',df_resultats.T['Nbre Loosers'].sum())
print('Cumulated gains',round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum() * _rate,2))
print('Cumulated losses',round(df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum() * _rate,2))
print(col.Fore.BLUE,'PROFIT FACTOR : ',\
    abs(round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum()/df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2)),col.Style.RESET_ALL)
print(col.Fore.CYAN,'Winners Ratio :',\
    round((df_resultats.T['Nbre Winners'].sum()*100)/(df_resultats.T['Nbre Loosers'].sum()+df_resultats.T['Nbre Winners'].sum()),2),'%',col.Style.RESET_ALL)
print('Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate,2))
print('Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate,2))
print('Average pnl',round(sum(PNL_REAL)/sum(TOTAL_OPEN) * _rate,2))
print('% Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate * 100 / _cash_ini,2))
print('% Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate / _cash_ini * 100,2))
print('% Average pnl',round((sum(PNL_REAL)/len(set(PNL_REAL)) * _rate) / _cash_ini * 100,2))
print('Number of opened trades',sum(TOTAL_OPEN))
print('Number of closed trades',sum(TOTAL_CLOSE))
#print("Nombre de positions en l'air",(sum(TOTAL_OPEN) - sum(TOTAL_CLOSE)))

 EUR/USD  results 
 Tested Period 2018-01-01 00:00:01  à 2018-12-31 23:59:59 
 Total Number of trades 5540 
Started Cash : 5000
P&L in currency:  -37711.6$ 
P&L in %:  -18.86% 
Average trade duration 225.36
# Winners  1727
# Loosers  3813
Cumulated gains 18041.75
Cumulated losses -55753.35
 PROFIT FACTOR :  0.32 
 Winners Ratio : 31.17 % 
Average Winners 3.26
Average Loosers -10.06
Average pnl -6.81
% Average Winners 0.0
% Average Loosers -0.01
% Average pnl -0.73
Number of opened trades 5540
Number of closed trades 5540


In [18]:
#########
##### ONE SHOT
#########

In [19]:
%%time

number = 10000
_period = 'm5'
_period2 = 'H1'
_period3 = 'D1'
_total = 0
_cash_ini = 200000
_cash = _cash_ini
_rate = 1
#_flag = 0

DATE = []
CONTRACT = []
OPEN_POZ = []
CLOSE_POZ = []
RATE_OPEN_POZ = []
RATE_CLOSE_POZ = []
PNL_LAT = []
PNL_REAL = []
TOTAL_OPEN = []
TOTAL_CLOSE = []
PRICE_BUY = []
PRICE_SELL = []
DER_POZ = []
TOTAL_PNL_LAT = []
TOTAL_PNL_REAL = []
_cash = _cash_ini
WINNERS = []
LOOSERS = []
SIGNAL = []
OPEN = []
OPEN_BID = []
OPEN_ASK = []

_verbose = 1

engine = pyttsx3.init()


%matplotlib inline
plt.style.use('seaborn')

df_resultats = pd.DataFrame(index=['TimeFrame 1','TimeFrame2','Equity','Nbre Winners','Nbre Loosers','Max lenght of trade','Min lenght of trade',\
    'Average lenght of trade','Cumul pnl'])

x = 'EUR/USD'

print('Prêt')


##########################################################
##########################################################
##########################################################

engine.say("קדימה")
engine.runAndWait()

position = 0

print(col.Fore.MAGENTA,'Le rate du ticker',x,'est à ',_rate,col.Style.RESET_ALL)

print('Chargement des bases')

data = pd.read_csv('Base_Clean/'+x.replace('/','')+'m5.csv')
data2 = pd.read_csv('Base_Clean/'+x.replace('/','')+'D1.csv')

data.index = pd.to_datetime(data['Date'])
data = data.drop(['Date','Total'],axis=1)
data.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data2['Date'] = data2['Date'].shift(-1)
data2.index = pd.to_datetime(data2['Date'])
data2 = data2.drop(['Date','Total'],axis=1)
data2.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data = data[(data.index >= _year_bottom) & (data.index <= _year_top)]
data2 = data2[(data2.index >= _year_bottom) & (data2.index <= _year_top)]

price = data.copy()
price2 = data2.copy()

print('Bases chargées')
print('Préparation des dataframes')

if '/JPY' in x:
    _pip_factor = 100
else:
    _pip_factor = 10000
price2['Range_Bid'] = (price2.Open_Bid.shift(1) - price2.Close_Bid.shift(1)) * _pip_factor
price2['Range_Ask'] = (price2.Open_Ask.shift(1) - price2.Close_Ask.shift(1)) * _pip_factor
price2['Range'] = (price2.Open.shift(1) - price2.Close.shift(1)) * _pip_factor
price2['OpenBid_Shift'] = price2['Open_Bid'].shift(-1)
price2['OpenAsk_Shift'] = price2['Open_Ask'].shift(-1)
price2['Open_Shift'] = price2['Open'].shift(-1)
price2['Signal'] = np.where((price2.Range_Ask >= _atr) | (price2.Range_Bid >= _atr) | (price2.Range >= _atr),1,\
    np.where((price2.Range_Ask <= -_atr) | (price2.Range_Bid <= -_atr) | (price2.Range <= -_atr),-1,0))
price2['TimeRange']= [price2.index[i].strftime(format='%Y-%m-%d') for i in range(len(price2))]
price['TimeRange']= [price.index[i].strftime(format='%Y-%m-%d') for i in range(len(price))]

print('TETEL process effectué')

print(col.Fore.CYAN,'ENTERING THE BACKTEST',col.Style.RESET_ALL)

for i in range(len(price)):
    try:
        SIGNAL.append(price2[price2.TimeRange == price.TimeRange[i]].Signal.values[0])
        OPEN.append(price2[price2.TimeRange == price.TimeRange[i]].Open_Shift.values[0])
        OPEN_BID.append(price2[price2.TimeRange == price.TimeRange[i]].OpenBid_Shift.values[0])
        OPEN_ASK.append(price2[price2.TimeRange == price.TimeRange[i]].OpenAsk_Shift.values[0])
    except:
        SIGNAL.append(999)
        OPEN.append(999)
        OPEN_BID.append(999)
        OPEN_ASK.append(999)
price['Signal'] = SIGNAL
price['OpenBid_Shift'] = OPEN_BID
price['OpenAsk_Shift'] = OPEN_ASK
price['Open_Shift'] = OPEN

price = price[(price.Signal != 999) & (price.OpenBid_Shift != 999) & (price.OpenAsk_Shift != 999)]
price = price.dropna()

_position = 0
_equity = 0
_nbtransactions = 0
backtest_graph = pd.DataFrame()
EQUITY = [_cash]
CASH = [_cash]
_winner = 0
_looser = 0
_index_entry = 0
TRADE_DURATION = []
_average_duration = 0
_size = 5000

PRICE_BUY = []
PRICE_SELL = []

_total = 0

_target = 0.006
_sl = 0.0015
_spread_filter = 0.00050
_filtre_multipoz = 0.5 # En %
_filtre_multipoz = _filtre_multipoz/100

_open_buy = 0
_open_sell = 0
_minipoz = 0
_flag = price[price.TimeRange[300]].TimeRange[0]
_stop = 0

for i in range(0,len(price)-300):

    if i == (len(price)-1) and _position != 0 :

        if _position == -1:
            _position = 999
            _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_sell
            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_sell,'position closed at',price.Close_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(-1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_sell)
            PRICE_SELL = []
            _open_sell = 0

        if _position == 1:

            _position = 999
            _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
            _total += _pnl
            _cash += _pnl
            _equity = _cash
            EQUITY.append(_equity)
            CASH.append(_cash)
            _looser += _open_buy

            TRADE_DURATION.append(i - _index_entry)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_buy,'positions closed at',price.Close_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(0)
            CLOSE_POZ.append(1)
            RATE_OPEN_POZ.append(0)
            RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
            PNL_LAT.append(0)
            PNL_REAL.append(_pnl)
            TOTAL_CLOSE.append(_open_buy) 
            TOTAL_PNL_LAT.append(0)
            TOTAL_PNL_REAL.append(_pnl)
            PRICE_BUY = []
            _open_buy = 0

    if _position == -1 and _stop == 1 and _flag == price[price.TimeRange[i-300]].TimeRange[0]:
        _stop = 0
        _flag = price[price.TimeRange[i+300]].TimeRange[0]
        _position = 0
        _pnl = - (price.High_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
        _total += _pnl
        _cash += _pnl
        _equity = _cash
        EQUITY.append(_equity)
        CASH.append(_cash)
        if _pnl > 0 :
            _winner += _open_sell
        else:
            _looser += _open_sell

        TRADE_DURATION.append(i - _index_entry)
        if _verbose == 1:
            print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
            if _pnl > 0 :
                print(_open_sell,'positions closed at',price.High_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
            else:
                print(_open_sell,'positions closed at',price.High_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
            print('nombre de candles en position :',i - _index_entry)
            print('Equity :', _equity)

        DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
        CONTRACT.append(x)
        OPEN_POZ.append(0)
        CLOSE_POZ.append(-1)
        RATE_OPEN_POZ.append(0)
        RATE_CLOSE_POZ.append(price.High_Ask.iloc[i])
        PNL_LAT.append(0)
        PNL_REAL.append(_pnl)
        TOTAL_PNL_LAT.append(0)
        TOTAL_PNL_REAL.append(_pnl)
        TOTAL_CLOSE.append(_open_sell)
        PRICE_SELL = [] 
        _open_sell = 0

    if _position == 1 and _stop == 1 and _flag == price[price.TimeRange[i-300]].TimeRange[0]:
        _stop = 0
        _flag = price[price.TimeRange[i+300]].TimeRange[0]
        _position = 0
        _pnl = (price.Low_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
        _total += _pnl
        _cash += _pnl
        _equity = _cash
        EQUITY.append(_equity)
        CASH.append(_cash)
        if _pnl > 0:
            _winner += _open_buy
        else:
            _looser += _open_buy

        TRADE_DURATION.append(i - _index_entry)
        if _verbose == 1:
            print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
            if _pnl > 0:
                print(_open_buy,'positions closed at',price.Low_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
            else:
                print(_open_buy,'positions closed at',price.Low_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
            print('nombre de candles en position :',i - _index_entry)
            print('Equity :', _equity)

        DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
        CONTRACT.append(x)
        OPEN_POZ.append(0)
        CLOSE_POZ.append(1)
        RATE_OPEN_POZ.append(0)
        RATE_CLOSE_POZ.append(price.Low_Bid.iloc[i])
        PNL_LAT.append(0)
        PNL_REAL.append(_pnl)
        TOTAL_PNL_LAT.append(0)
        TOTAL_PNL_REAL.append(_pnl)
        TOTAL_CLOSE.append(_open_buy)
        PRICE_BUY = [] 
        _open_buy = 0

    else:
        _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
        _equity = _cash + _pnl

        EQUITY.append(_equity)
        CASH.append(_cash)

        DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
        CONTRACT.append(x)
        OPEN_POZ.append(0)
        CLOSE_POZ.append(0)
        RATE_OPEN_POZ.append(0)
        RATE_CLOSE_POZ.append(0)
        PNL_LAT.append(_pnl)
        PNL_REAL.append(0)
        TOTAL_PNL_LAT.append(_pnl)

        TOTAL_PNL_REAL.append(0)

    if _position == 0 and _stop == 0 and _flag == price[price.TimeRange[i]].TimeRange[0]:
        # BUY SIGNAL
        if price.Signal[i] == -1 : # -1 and (price.OpenBid_Shift[i] - price.High_Bid[i]) * _pip_factor < -15 and \
            #(price.OpenBid_Shift[i] - price.High_Bid[i]) * _pip_factor > -35 :
            _stop = 1
            _flag = price[price.TimeRange[i]].TimeRange[0]
            _pnl = 0
            _open_buy += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _position = 1
            _index_entry = i
            _nbtransactions += 1
            price_buy = price.Close_Ask.iloc[i]
            PRICE_BUY.append(price_buy)
            _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 bought at', price_buy,'(verification liste',PRICE_BUY[-1],')')

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_buy)
            RATE_CLOSE_POZ.append(0)
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_OPEN.append(1) 
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0) 

        
        
        # SELL SIGNAL
        elif price.Signal[i] == 1 : # 1 and (price.OpenAsk_Shift[i] - price.Low_Ask[i]) * _pip_factor > 15 and\
            #(price.OpenAsk_Shift[i] - price.Low_Ask[i]) * _pip_factor < 35 :
            _stop = 1
            _flag = price[price.TimeRange[i]].TimeRange[0]
            _pnl = 0
            _open_sell += 1
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)
            _index_entry = i
            _position = -1
            _nbtransactions += 1
            price_sell = price.Close_Bid.iloc[i]
            PRICE_SELL.append(price_sell)
            _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
            if _verbose == 1:
                print(col.Fore.MAGENTA,price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'),col.Style.RESET_ALL)
                print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')

            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
            CONTRACT.append(x)
            OPEN_POZ.append(-1)
            CLOSE_POZ.append(0)
            RATE_OPEN_POZ.append(price_sell)
            RATE_CLOSE_POZ.append(0) 
            PNL_LAT.append(_pnl)
            PNL_REAL.append(0)
            TOTAL_PNL_LAT.append(_pnl)
            TOTAL_PNL_REAL.append(0)

            TOTAL_OPEN.append(1)

        else :
            _flag = price[price.TimeRange[i+300]].TimeRange[0]
            _pnl = 0
            _equity = _cash + _pnl
            EQUITY.append(_equity)
            CASH.append(_cash)

    
            
print('')
print('Fin du Backtest => Computing') 

try:
    _average_duration = round(statistics.mean(TRADE_DURATION),2)
    _max_duration = max(TRADE_DURATION)
    _min_duration = min([item for item in TRADE_DURATION if item !=0])

except:
    print("(pas de duration)") 
    _average_duration = 'NA' 
print(col.Fore.BLUE,'Pour le ticker',col.Fore.YELLOW,x,col.Style.RESET_ALL)
if _total > 0:              
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.GREEN,round(_total,2),'!. En ',\
        _nbtransactions,col.Style.RESET_ALL,' transactions.' )
else:
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.RED,round(_total,2),'!. En ',\
        _nbtransactions,col.Style.RESET_ALL,' transactions.' ) 
print(col.Fore.GREEN,"\nNbre de winners :",_winner,col.Style.RESET_ALL)
print(col.Fore.RED,"\nNbre de loosers :",_looser,col.Style.RESET_ALL)

backtest_graph['Equity'] = EQUITY

df_resultats[x] = [_period,_period2,(round(_equity,2)),(_winner),(_looser),(_max_duration),(_min_duration),(_average_duration),(_total)]

DER_POZ.append(_pnl)

engine.say("Finito caucau")
engine.runAndWait()


Prêt
 Le rate du ticker EUR/USD est à  1 
Chargement des bases
Bases chargées
Préparation des dataframes
TETEL process effectué
 ENTERING THE BACKTEST 
 2018-01-14 19:20:00 
Position 1 bought at 1.22061 (verification liste 1.22061 )
 2018-01-15 21:05:00 
1 positions closed at 1.22634  pnl 28.650000000000063 
nombre de candles en position : 300
Equity : 200028.65
 2018-01-25 00:00:00 
Position 1 bought at 1.23958 (verification liste 1.23958 )
 2018-01-28 22:05:00 
1 positions closed at 1.24153  pnl 9.750000000000592 
nombre de candles en position : 300
Equity : 200038.4
 2018-02-08 00:00:00 
Position 1 sold at 1.2272 (verification liste 1.2272 )
 2018-02-11 21:30:00 
1 positions closed at 1.22494  pnl 11.300000000000754 
nombre de candles en position : 300
Equity : 200049.69999999998
 2018-02-15 00:00:00 
Position 1 bought at 1.2468299999999999 (verification liste 1.2468299999999999 )
 2018-02-18 22:20:00 
1 positions closed at 1.24088  pnl -29.75000000000061 
nombre de candles en posit

In [20]:
############################################################
############################################################
############################################################

df_historical = pd.DataFrame()
df_historical = pd.DataFrame(index=DATE)
df_historical['Contract'] = CONTRACT
df_historical['Open_Poz'] = OPEN_POZ
df_historical['Close_Pos'] = CLOSE_POZ
df_historical['Rate_Open_Poz'] = RATE_OPEN_POZ 
df_historical['Rate_Close_Poze'] = RATE_CLOSE_POZ
df_historical['Pnl_Lat'] = TOTAL_PNL_LAT
df_historical['Pnl_Real'] = TOTAL_PNL_REAL
df_historical = df_historical.sort_index()
_generated_cash = round(df_historical.Pnl_Real.sum() * _rate,2)
_generated_cash_perc = round((_generated_cash / _cash_ini) * 100,2)
print(col.Fore.YELLOW,x,col.Fore.BLUE,'results',col.Style.RESET_ALL)
print(col.Fore.MAGENTA,'Tested Period',_year_bottom,' à',_year_top,col.Style.RESET_ALL)
print(col.Fore.CYAN,'Total Number of trades',max([sum(TOTAL_OPEN),sum(TOTAL_CLOSE)]),col.Style.RESET_ALL)
if _generated_cash <= 0:
    print('Started Cash :',_size)
    print('P&L in currency:',col.Fore.RED,str(_generated_cash)+'$',col.Style.RESET_ALL)
    print('P&L in %:',col.Fore.RED,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

else:
    print('Started Cash :',_size)
    print('P&L  in currency:',col.Fore.GREEN,str(_generated_cash)+'$',col.Style.RESET_ALL)
    print('P&L in %:',col.Fore.GREEN,str(_generated_cash_perc)+'%',col.Style.RESET_ALL)

#print('Pnl latent minimal :',str(round((df_historical.Pnl_Real.min()) * _rate,2))+'$')
#print('Pnl latent maximal :',str(round((df_historical.Pnl_Real.max())* _rate,2))+'$')
#print('Pnl latent moyen :',str(round(df_historical.Pnl_Real.mean()* _rate,2))+'$')
#print('Maximal trade duration',_max_duration)
#print('Minimal trade duration',_min_duration)
print('Average trade duration',_average_duration)
print('# Winners ',df_resultats.T['Nbre Winners'].sum())
print('# Loosers ',df_resultats.T['Nbre Loosers'].sum())
print('Cumulated gains',round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum() * _rate,2))
print('Cumulated losses',round(df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum() * _rate,2))
print(col.Fore.BLUE,'PROFIT FACTOR : ',\
    abs(round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum()/df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2)),col.Style.RESET_ALL)
print(col.Fore.CYAN,'Winners Ratio :',\
    round((df_resultats.T['Nbre Winners'].sum()*100)/(df_resultats.T['Nbre Loosers'].sum()+df_resultats.T['Nbre Winners'].sum()),2),'%',col.Style.RESET_ALL)
print('Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate,2))
print('Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate,2))
print('Average pnl',round(sum(PNL_REAL)/sum(TOTAL_OPEN) * _rate,2))
print('% Average Winners',round(sum(list(filter(lambda x:  x > 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate * 100 / _cash_ini,2))
print('% Average Loosers',round(sum(list(filter(lambda x:  x < 0,PNL_REAL)))/sum(TOTAL_OPEN) * _rate / _cash_ini * 100,2))
print('% Average pnl',round((sum(PNL_REAL)/len(set(PNL_REAL)) * _rate) / _cash_ini * 100,2))
print('Number of opened trades',sum(TOTAL_OPEN))
print('Number of closed trades',sum(TOTAL_CLOSE))
#print("Nombre de positions en l'air",(sum(TOTAL_OPEN) - sum(TOTAL_CLOSE)))

 EUR/USD  results 
 Tested Period 2018-01-01 00:00:01  à 2018-12-31 23:59:59 
 Total Number of trades 21 
Started Cash : 5000
P&L in currency:  -163.6$ 
P&L in %:  -0.08% 
Average trade duration 300
# Winners  10
# Loosers  11
Cumulated gains 143.7
Cumulated losses -307.3
 PROFIT FACTOR :  0.47 
 Winners Ratio : 47.62 % 
Average Winners 6.84
Average Loosers -14.63
Average pnl -7.79
% Average Winners 0.0
% Average Loosers -0.01
% Average pnl -0.0
Number of opened trades 21
Number of closed trades 21


In [21]:
i

61352

In [90]:
price2.head(1)

,Open_Bid,High_Bid,Low_Bid,Close_Bid,Open_Ask,High_Ask,Low_Ask,Close_Ask,Open,High,Low,Close,Range_Bid,Range_Ask,Range,OpenBid_Shift,OpenAsk_Shift,Open_Shift,Signal,TimeRange
Date,,,,,,,,,,,,,,,,,,,,
2020-01-01 22:00:00,1.11979,1.12387,1.11959,1.12217,1.11994,1.12399,1.11992,1.12234,1.119865,1.12393,1.119755,1.122255,NaN,NaN,NaN,1.12217,1.12234,1.122255,0,2020-01-01


In [ ]:
_year_bottom = '2010-01-01 00:00:01'
_year_top = '2020-12-31 23:59:59'

data2 = pd.read_csv('Base_Clean/'+x.replace('/','')+'D1.csv')

data2['Date'] = data2['Date'].shift(-1)
data2.index = pd.to_datetime(data2['Date'])
data2 = data2.drop(['Date','Total'],axis=1)
data2.rename(columns={'OpenBid':'Open_Bid','HighBid':'High_Bid','LowBid':'Low_Bid','CloseBid':'Close_Bid',\
    'OpenAsk':'Open_Ask','HighAsk':'High_Ask','LowAsk':'Low_Ask','CloseAsk':'Close_Ask'},inplace = True)

data2 = data2[(data2.index >= _year_bottom) & (data2.index <= _year_top)]

price2 = data2.copy()

if '/JPY' in x:
    _pip_factor = 100
else:
    _pip_factor = 10000
price2['Range_Bid'] = (price2.Open_Bid.shift(1) - price2.Close_Bid.shift(1)) * _pip_factor
price2['Range_Ask'] = (price2.Open_Ask.shift(1) - price2.Close_Ask.shift(1)) * _pip_factor
price2['Range'] = (price2.Open.shift(1) - price2.Close.shift(1)) * _pip_factor

price2['Signal'] = np.where((price2.Range_Ask >= 80) | (price2.Range_Bid >= 80) | (price2.Range >= 80),1,\
    np.where((price2.Range_Ask <= -80) | (price2.Range_Bid <= -80) | (price2.Range <= -80),-1,0))
price2['TimeRange']= [price2.index[i].strftime(format='%Y-%m-%d') for i in range(len(price2))]

pattern_data=[0,-1,-1,0]
data=list(price2.Signal)

count=0
for i in range(0,len(data),1):
    if data[i:i+len(pattern_data)]==pattern_data:
        print(i,data[i:i+len(pattern_data)])
        count+=1

print(count)